
# CS436 Project: 3D Scene Reconstruction
## Week 3 (Rewritten): Pairwise Clouds -> Unified Point Cloud

This version pushes the stable Week 2 two-view pipeline harder: denser matching, stricter RANSAC, pose refinement with PnP, outlier filtering, per-pair PLYs under `ply_pairs/`, and a merged cloud. Incremental SfM is intentionally skipped.



**Pipeline**
- Load images from `../data` (optionally downscale) and compute intrinsics from image size.
- Build pairs (consecutive by default; optionally all pairs, optional loop closure).
- For each pair: dense SIFT+ORB, Essential matrix (tight RANSAC), recover pose, triangulate, filter, refine with PnP, save `ply_pairs/pair_XX_YY.ply`.
- Chain consecutive poses to place pair clouds in a shared frame; merge and save `ply_pairs/reconstruction_pairs_merged.ply` (with optional voxel + statistical filtering).
- Optional Open3D viewer for the merged cloud.


In [1]:

import os, sys, gc
import numpy as np
import cv2

PROJECT_ROOT = os.path.abspath(os.path.join('..'))
SRC_DIR = os.path.join(PROJECT_ROOT, 'src')
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

from utils import (
    load_images_from_folder,
    get_dense_combined_matches,
    sample_colors,
    save_colored_point_cloud_to_ply,
    save_point_cloud_to_ply,
    set_global_memory_mode,
)
from reconstruction import (
    get_intrinsic_matrix,
    calculate_essential_matrix_from_points,
    recover_camera_pose,
    triangulate_points,
)

set_global_memory_mode()
print('Imports ready; memory mode on.')


Imports ready; memory mode on.


In [2]:

# Configuration
DATA_DIR = '../data'
PAIR_OUTPUT_DIR = 'ply_pairs'   # keep outputs inside notebooks/
MAX_VIEWS = None                # set int to limit number of images (e.g., 20)
DOWNSCALE_MAX_SIDE = 1280       # None to disable; tradeoff: more detail vs RAM
USE_ALL_PAIRS = False           # True = every combination; False = consecutive only
CLOSE_LOOP = True              # if True and consecutive mode, also add (last, first)

# Reconstruction tunables
RANSAC_THRESH = 1.5             # px threshold for Essential matrix
MIN_COMBINED_MATCHES = 40
MIN_INLIERS = 25
MIN_CHEIRALITY = 15
MAX_DIST_PERCENTILE = 98        # drop extreme far points
VOXEL_SIZE = 0.0                # set >0 (e.g., 0.01) to downsample merged cloud
OUTLIER_STD = 2.0               # statistical outlier removal std ratio

os.makedirs(PAIR_OUTPUT_DIR, exist_ok=True)
print(f'Pair outputs will be saved under {PAIR_OUTPUT_DIR}/')


Pair outputs will be saved under ply_pairs/


In [3]:

# Load images
images = load_images_from_folder(DATA_DIR, target_width=DOWNSCALE_MAX_SIDE or 1024)
if MAX_VIEWS is not None:
    images = images[:MAX_VIEWS]
print(f'Total images loaded: {len(images)}')
assert len(images) >= 2, 'Need at least two images to reconstruct.'

K = get_intrinsic_matrix(images[0].shape)
print('Intrinsic matrix (approx):')
print(K)


Successfully loaded 33 images from ../data
Images resized to max width: 1280px
Total images loaded: 33
Intrinsic matrix (approx):
[[1280.    0.  640.]
 [   0. 1280.  853.]
 [   0.    0.    1.]]


In [4]:

# Build pair list
pairs = []
num_images = len(images)
if USE_ALL_PAIRS:
    for i in range(num_images):
        for j in range(i+1, num_images):
            pairs.append((i, j))
else:
    for i in range(num_images - 1):
        pairs.append((i, i+1))
    if CLOSE_LOOP and num_images > 2:
        pairs.append((num_images-1, 0))

print(f'Planned pairs: {len(pairs)}')
print(pairs)


Planned pairs: 33
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (24, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30), (30, 31), (31, 32), (32, 0)]


In [5]:
# Helpers

def filter_points(points_3d, colors=None, max_percentile=98):
    if len(points_3d) == 0:
        return points_3d, colors, np.ones(0, dtype=bool)
    mask = np.isfinite(points_3d).all(axis=1)
    mask &= points_3d[:, 2] > 0
    if mask.any() and max_percentile is not None:
        dists = np.linalg.norm(points_3d, axis=1)
        thr = np.percentile(dists[mask], max_percentile)
        mask &= dists <= thr
    pts = points_3d[mask]
    cols = colors[mask] if colors is not None else None
    return pts, cols, mask


def reconstruct_pair(img1, img2, K, idx1, idx2):
    print(f'Pair {idx1} -> {idx2}')
    pts1_raw, pts2_raw, stats = get_dense_combined_matches(
        img1, img2,
        sift_nfeatures=15000,
        sift_ratio=0.80,
        orb_nfeatures=8000,
        orb_ratio=0.80,
        subpixel=True,
    )
    print(f"  Matches: SIFT={stats['sift_matches']} ORB={stats['orb_matches']} Combined={stats['combined_matches']}")
    if len(pts1_raw) < MIN_COMBINED_MATCHES:
        print('  Skipped: too few matches.')
        return None, None, None

    E, mask, pts1, pts2 = calculate_essential_matrix_from_points(
        pts1_raw, pts2_raw, K, ransac_thresh=RANSAC_THRESH, prob=0.999)
    mask_b = mask.ravel() > 0 if mask is not None else np.zeros(len(pts1), dtype=bool)
    pts1_in, pts2_in = pts1[mask_b], pts2[mask_b]
    print(f'  RANSAC inliers: {len(pts1_in)}')
    if len(pts1_in) < MIN_INLIERS:
        print('  Skipped: insufficient inliers.')
        return None, None, None

    R, t, pose_mask = recover_camera_pose(E, pts1_in, pts2_in, K)
    pose_b = pose_mask.ravel() > 0 if pose_mask is not None else np.zeros(len(pts1_in), dtype=bool)
    pts1_final, pts2_final = pts1_in[pose_b], pts2_in[pose_b]
    print(f'  Cheirality passed: {len(pts1_final)}')
    if len(pts1_final) < MIN_CHEIRALITY:
        print('  Skipped: cheirality rejected too many points.')
        return None, None, None

    # Normalize translation scale to 1 to reduce drift across chained poses
    t_norm = np.linalg.norm(t)
    if t_norm > 0:
        t = t / t_norm

    points_3d = triangulate_points(R, t, K, pts1_final, pts2_final)
    colors = 0.5 * (sample_colors(img1, pts1_final).astype(np.float32) +
                    sample_colors(img2, pts2_final).astype(np.float32))

    points_3d, colors, keep_mask = filter_points(points_3d, colors, max_percentile=MAX_DIST_PERCENTILE)
    pts1_final = pts1_final[keep_mask]
    pts2_final = pts2_final[keep_mask]
    if colors is not None:
        colors = colors.astype(np.uint8)

    if len(points_3d) == 0:
        print('  Skipped: no finite points after filtering.')
        return None, None, None

    # Pose refinement with PnP
    success, rvec, tvec, _ = cv2.solvePnPRansac(
        points_3d.astype(np.float32), pts2_final.astype(np.float32), K, None,
        iterationsCount=300, reprojectionError=3.0, confidence=0.999,
        flags=cv2.SOLVEPNP_ITERATIVE,
    )
    if success:
        R_refined, _ = cv2.Rodrigues(rvec)
        t_refined = tvec.reshape(3,1)
        t_ref_norm = np.linalg.norm(t_refined)
        if t_ref_norm > 0:
            t_refined = t_refined / t_ref_norm
        # Re-triangulate with refined pose for better alignment
        points_3d = triangulate_points(R_refined, t_refined, K, pts1_final, pts2_final)
        points_3d, colors, keep_mask = filter_points(points_3d, colors, max_percentile=MAX_DIST_PERCENTILE)
        pts1_final = pts1_final[keep_mask]
        pts2_final = pts2_final[keep_mask]
        pose_out = (R_refined, t_refined)
        print(f'  Pose refined via PnP (inliers: {len(points_3d)})')
    else:
        pose_out = (R, t)
        print('  Pose refinement skipped (PnP failed).')

    if colors is not None:
        colors = colors.astype(np.uint8)
    print(f'  Reconstructed points: {len(points_3d)}')
    return points_3d, colors, pose_out


In [6]:

# Run pairwise reconstruction and save per-pair PLYs
all_points = []
all_colors = []
camera_poses = {}  # idx -> (R_global, t_global)
summary = []

for idx, (i, j) in enumerate(pairs):
    pts3d, cols, pose = reconstruct_pair(images[i], images[j], K, i, j)
    base = f'pair_{i:02d}_{j:02d}'
    ply_path = os.path.join(PAIR_OUTPUT_DIR, base + '.ply')

    if pts3d is not None and cols is not None:
        try:
            save_colored_point_cloud_to_ply(pts3d, cols, ply_path)
        except Exception:
            save_point_cloud_to_ply(pts3d, ply_path)
        print(f'  Saved {ply_path} ({len(pts3d)} pts).')
        summary.append(((i, j), len(pts3d)))
    else:
        print(f'  No output for pair {i}-{j}.')
        summary.append(((i, j), 0))
        continue

    # Chain poses only for consecutive pairs
    if not USE_ALL_PAIRS:
        if i == 0 and j == 1 and (0 not in camera_poses):
            R_rel, t_rel = pose
            camera_poses[0] = (np.eye(3), np.zeros((3,1)))
            camera_poses[1] = (R_rel, t_rel)
            pts_global = (np.eye(3) @ pts3d.T).T  # reference frame at cam0
            all_points.append(pts_global)
            all_colors.append(cols)
        elif i in camera_poses and j == i + 1:
            R_rel, t_rel = pose
            R_i, t_i = camera_poses[i]
            R_j = R_i @ R_rel
            t_j = R_i @ t_rel + t_i
            camera_poses[j] = (R_j, t_j)
            pts_global = (R_i @ pts3d.T).T + t_i.T
            all_points.append(pts_global)
            all_colors.append(cols)
        elif CLOSE_LOOP and i == num_images - 1 and j == 0 and i in camera_poses:
            R_rel, t_rel = pose
            R_i, t_i = camera_poses[i]
            R_j = R_i @ R_rel
            t_j = R_i @ t_rel + t_i
            camera_poses[0] = (R_j, t_j)
            pts_global = (R_i @ pts3d.T).T + t_i.T
            all_points.append(pts_global)
            all_colors.append(cols)

print('Pair summary (pair -> points):')
for (i, j), n in summary:
    print(f'  ({i},{j}) -> {n}')

print(f'Camera poses recovered: {len(camera_poses)}/{num_images}')


Pair 0 -> 1
  Matches: SIFT=1791 ORB=1869 Combined=3660
  Matches: SIFT=1791 ORB=1869 Combined=3660
  RANSAC inliers: 2150
  RANSAC inliers: 2150
  Cheirality passed: 2149
  Cheirality passed: 2149
  Pose refined via PnP (inliers: 2063)
  Reconstructed points: 2063
Saved colored point cloud to ply_pairs\pair_00_01.ply
  Saved ply_pairs\pair_00_01.ply (2063 pts).
Pair 1 -> 2
  Pose refined via PnP (inliers: 2063)
  Reconstructed points: 2063
Saved colored point cloud to ply_pairs\pair_00_01.ply
  Saved ply_pairs\pair_00_01.ply (2063 pts).
Pair 1 -> 2
  Matches: SIFT=1566 ORB=1393 Combined=2959
  Matches: SIFT=1566 ORB=1393 Combined=2959
  RANSAC inliers: 1993
  RANSAC inliers: 1993
  Cheirality passed: 1993
  Cheirality passed: 1993
  Pose refined via PnP (inliers: 1913)
  Reconstructed points: 1913
Saved colored point cloud to ply_pairs\pair_01_02.ply
  Saved ply_pairs\pair_01_02.ply (1913 pts).
Pair 2 -> 3
  Pose refined via PnP (inliers: 1913)
  Reconstructed points: 1913
Saved color

In [7]:

# Merge all saved pair clouds into one PLY (chained poses preferred when available)
MERGED_PLY = os.path.join(PAIR_OUTPUT_DIR, 'reconstruction_pairs_merged.ply')

try:
    import open3d as o3d
except ImportError:
    o3d = None

if all_points:
    merged_pts = np.vstack(all_points)
    merged_cols = np.vstack(all_colors) if all_colors else None
    if merged_cols is None:
        save_point_cloud_to_ply(merged_pts, MERGED_PLY)
    else:
        save_colored_point_cloud_to_ply(merged_pts, merged_cols, MERGED_PLY)
    print(f'Merged from chained poses: {MERGED_PLY} with {len(merged_pts)} pts.')
elif o3d is None:
    print('Open3D not installed; cannot merge PLYs on disk. Install open3d or rerun with chained poses.')
else:
    ply_files = [f for f in os.listdir(PAIR_OUTPUT_DIR) if f.lower().endswith('.ply') and f.startswith('pair_')]
    if not ply_files:
        raise RuntimeError('No pair PLYs found to merge.')
    pts_list, col_list = [], []
    for f in sorted(ply_files):
        path = os.path.join(PAIR_OUTPUT_DIR, f)
        pcd = o3d.io.read_point_cloud(path)
        pts = np.asarray(pcd.points)
        if pts.size == 0:
            continue
        pts_list.append(pts)
        if pcd.has_colors():
            col_list.append(np.asarray(pcd.colors))
        else:
            col_list.append(np.full((pts.shape[0], 3), 0.5, dtype=np.float64))
    merged_pts = np.vstack(pts_list)
    merged_cols = np.vstack(col_list)
    pcd_merged = o3d.geometry.PointCloud()
    pcd_merged.points = o3d.utility.Vector3dVector(merged_pts)
    pcd_merged.colors = o3d.utility.Vector3dVector(merged_cols)

    if VOXEL_SIZE and VOXEL_SIZE > 0:
        pcd_merged = pcd_merged.voxel_down_sample(VOXEL_SIZE)
    if OUTLIER_STD and OUTLIER_STD > 0:
        cl, ind = pcd_merged.remove_statistical_outlier(nb_neighbors=40, std_ratio=OUTLIER_STD)
        pcd_merged = cl

    o3d.io.write_point_cloud(MERGED_PLY, pcd_merged, write_ascii=True)
    print(f'Merged by concatenation: {MERGED_PLY} with {len(pcd_merged.points)} pts.')


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Saved colored point cloud to ply_pairs\reconstruction_pairs_merged.ply
Merged from chained poses: ply_pairs\reconstruction_pairs_merged.ply with 42865 pts.
Saved colored point cloud to ply_pairs\reconstruction_pairs_merged.ply
Merged from chained poses: ply_pairs\reconstruction_pairs_merged.ply with 42865 pts.


In [10]:
# Optional: preview merged cloud
try:
    import open3d as o3d
    if os.path.exists(MERGED_PLY):
        pcd = o3d.io.read_point_cloud(MERGED_PLY)
        if len(np.asarray(pcd.points)) == 0:
            print('Merged cloud is empty.')
        else:
            bbox = pcd.get_axis_aligned_bounding_box(); bbox.color = (1, 0.6, 0)
            origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)
            print(f'Launching Open3D viewer with {len(pcd.points)} points...')
            o3d.visualization.draw_geometries([pcd, bbox, origin])
    else:
        print('Merged PLY not found; run merge cell first.')
except ImportError:
    print('Open3D not installed. Skipping visualization.')


Launching Open3D viewer with 42865 points...


In [9]:

# Optional: Auto-refine alignment of all pair clouds using ICP registration
print("\\n=== ICP-Based Auto-Alignment (Chained) ===")
try:
    import open3d as o3d
    import glob
    
    # Find all pair PLYs in order
    ply_files = sorted([f for f in glob.glob(os.path.join(PAIR_OUTPUT_DIR, 'pair_*.ply'))])
    
    if not ply_files:
        print("No pair PLYs found. Run the reconstruction cell first.")
    else:
        print(f"Found {len(ply_files)} pair clouds to align.")
        
        # Load all clouds
        clouds = []
        for f in ply_files:
            pcd = o3d.io.read_point_cloud(f)
            if len(np.asarray(pcd.points)) > 0:
                clouds.append(pcd)
                print(f"  Loaded {os.path.basename(f)}: {len(pcd.points)} points")
            else:
                print(f"  Skipped {os.path.basename(f)} (empty)")
        
        if len(clouds) < 2:
            print("Not enough non-empty clouds to align.")
        else:
            # Chain ICP: align each cloud to the previous one
            print("\\nStarting chained ICP alignment...")
            transformations = [np.eye(4)]  # First cloud is reference
            
            for i in range(1, len(clouds)):
                src = clouds[i]
                ref = clouds[i-1]
                
                # ICP registration
                reg_result = o3d.pipelines.registration.registration_icp(
                    src, ref,
                    max_correspondence_distance=0.5,  # Adjust if clouds are far apart
                    estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(),
                    criteria=o3d.pipelines.registration.ICPConvergenceCriteria(
                        relative_fitness=1e-6,
                        relative_rmse=1e-6,
                        max_iteration=50
                    )
                )
                
                # Apply transformation to current cloud
                src.transform(reg_result.transformation)
                transformations.append(reg_result.transformation)
                
                fitness = reg_result.fitness
                rmse = reg_result.inlier_rmse
                print(f"  Pair {i-1}→{i}: fitness={fitness:.4f}, RMSE={rmse:.6f}")
            
            # Merge all aligned clouds
            print("\\nMerging aligned clouds...")
            merged_icp = o3d.geometry.PointCloud()
            for cloud in clouds:
                merged_icp += cloud
            
            # Optional: downsample merged cloud
            if VOXEL_SIZE and VOXEL_SIZE > 0:
                merged_icp = merged_icp.voxel_down_sample(VOXEL_SIZE)
                print(f"Downsampled to {len(merged_icp.points)} points (voxel_size={VOXEL_SIZE}).")
            
            # Optional: statistical outlier removal
            if OUTLIER_STD and OUTLIER_STD > 0:
                cl, ind = merged_icp.remove_statistical_outlier(nb_neighbors=40, std_ratio=OUTLIER_STD)
                merged_icp = cl
                print(f"Removed outliers, {len(merged_icp.points)} points remain.")
            
            # Save refined merged cloud
            ICP_MERGED_PLY = os.path.join(PAIR_OUTPUT_DIR, 'reconstruction_icp_refined.ply')
            o3d.io.write_point_cloud(ICP_MERGED_PLY, merged_icp, write_ascii=True)
            print(f"\\n✅ Saved ICP-refined merged cloud: {ICP_MERGED_PLY}")
            print(f"   Total points: {len(merged_icp.points)}")
            
            # Visualize result
            print("\\nLaunching Open3D viewer for ICP-refined cloud...")
            bbox = merged_icp.get_axis_aligned_bounding_box()
            bbox.color = (1, 0.6, 0)
            origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)
            o3d.visualization.draw_geometries([merged_icp, bbox, origin], window_name="ICP-Refined Merged Cloud")
            
except ImportError:
    print("⚠️  Open3D not installed. Skipping ICP refinement. Install with: pip install open3d")
except Exception as e:
    print(f"⚠️  ICP alignment error: {e}")


\n=== ICP-Based Auto-Alignment (Chained) ===
Found 33 pair clouds to align.
  Loaded pair_00_01.ply: 2063 points
  Loaded pair_01_02.ply: 1913 points
  Loaded pair_02_03.ply: 1527 points
  Loaded pair_03_04.ply: 1722 points
  Loaded pair_04_05.ply: 1765 points
  Loaded pair_05_06.ply: 27 points
  Loaded pair_06_07.ply: 2133 points
  Loaded pair_07_08.ply: 1932 points
  Loaded pair_08_09.ply: 1807 points
  Loaded pair_09_10.ply: 1502 points
  Loaded pair_10_11.ply: 1805 points
  Loaded pair_11_12.ply: 1432 points
  Loaded pair_12_13.ply: 1862 points
  Loaded pair_13_14.ply: 1768 points
  Loaded pair_14_15.ply: 1501 points
  Loaded pair_15_16.ply: 169 points
  Loaded pair_16_17.ply: 883 points
  Loaded pair_17_18.ply: 1458 points
  Loaded pair_18_19.ply: 1447 points
  Loaded pair_19_20.ply: 1109 points
  Loaded pair_20_21.ply: 1390 points
  Loaded pair_21_22.ply: 460 points
  Loaded pair_22_23.ply: 143 points
  Loaded pair_23_24.ply: 1881 points
  Loaded pair_24_25.ply: 1636 points
  Loa